# Order Imbalance Strategy in Chinese Future Markets
### Notebook by samyu@defytrading.com
### Support by Defytrading
# Introduction
This project is to replicate the paper from D.Shen(2015), who analysed the factor 'Order imbalance' to have a significant impact on the change in CSI300 Futures tick data backed in 2014. Here are his findings:

1. Developed a trading strategy by fitting the order imbalance factor model to forecast 10 seconds mid price change.
2. Model shows high profitability when trading signal is greater than 0.2 ticks.
3. Profits made by the strategy are strongly correlated with the total trading volume in the market

Additional findings:
1. Improved trading signal by extending model to 2 more factors : Order imbalance ratio and mid price basis mean reversion, and divided variables by bid-ask spread. 
2. Large spread indicates low price change
3. Trading parameters optimization : Forecast window for average change and trading threshold were closer to 5 and 0.15 respectively

Results from D.Shen(2015):
1. Average Correlation coefficeint of OI and instantaneous mid-price change : 0.3935
2. Fitting a linear regression model gives average daily R-squared : 0.155
3. Correlation between 10s interval between price change and VOI R-squared: 0.6537
4. Variable samples proved to be stationary by ADF/KPSS test

# Step 1 : Identifying Objectives
The first step of this project is to define what we are looking for. In my own understanding, the whole research can guide us to potential trading strategy and we are about to find out whether the key factor fits well with the data. D.Shen(2015) has proven a significant amount of profit backing up by the linear model. That means the signal(alpha) has a robust performance in the CSI300 Future tick data. The next step is to perform data cleaning. It is crucially important because uncleaned data may lead to irregular results.

# Step 2 : Checking & tidying data
Facts about the SHFE tick data set from BGT:
1. Tick data from 00:00:00.000 to 23:59:59.500 GMT+8
2. Interval per each entry : 5ms difference(Snapshot data)
3. 16 Future contracts with 12 expiries & Multiple Option contracts(Not included in this research). e.g. RB1912 to RB2012
4. LEVEL 2 quote.
5. Volume and Turnover resets at 21:00:00.000 for all contracts, which means, volume and turnover counts from previous day's 21:00:00.000 for contracts that trades during night session.

Let us import the dataset and see what its looks like:

In [105]:
import numpy as np
import os
import pandas as pd
import statsmodels.api as sm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#read file
df = pd.read_csv('ShfeMulti_20191031.csv', index_col=None, header=None)
df.columns = ['Timestamp','Contract', 'time','last_price','volume','turnover','open_interest', #rename column
              'BP1','BV1','AP1','AV1',
              'BP2','BV2','AP2','AV2',
              'BP3','BV3','AP3','AV3',
              'BP4','BV4','AP4','AV4',
              'BP5','BV5','AP5','AV5'] 
df.head(10)

,Timestamp,Contract,time,last_price,volume,turnover,open_interest,BP1,BV1,AP1,AV1,BP2,BV2,AP2,AV2,BP3,BV3,AP3,AV3,BP4,BV4,AP4,AV4,BP5,BV5,AP5,AV5
0,3240447216.64,ag1911,20:59:00.500,4321.00,0.00,0.00,324.00,4282.00,2.00,4396.00,1.00,4246.00,1.00,4424.00,2.00,4216.00,2.00,0.00,0.00,4135.00,2.00,0.00,0.00,4078.00,1.00,0.00,0.00
1,3240447216.64,ag1912,20:59:00.500,4359.00,4224.00,276186240.00,815008.00,4358.00,41.00,4359.00,232.00,4357.00,44.00,4360.00,280.00,4356.00,288.00,4361.00,387.00,4355.00,181.00,4362.00,212.00,4354.00,43.00,4363.00,220.00
2,3240447216.65,ag2001,20:59:00.500,4377.00,0.00,0.00,2386.00,4360.00,1.00,4397.00,10.00,4358.00,6.00,4398.00,6.00,4357.00,10.00,4399.00,1.00,4334.00,1.00,4461.00,1.00,4319.00,1.00,4470.00,1.00
3,3240447216.65,ag2002,20:59:00.500,4390.00,274.00,18042900.00,339136.00,4388.00,6.00,4391.00,6.00,4387.00,1.00,4392.00,7.00,4386.00,1.00,4393.00,7.00,4385.00,8.00,4394.00,13.00,4384.00,15.00,4395.00,88.00
4,3240447216.65,ag2003,20:59:00.500,4405.00,0.00,0.00,490.00,4358.00,1.00,4500.00,1.00,4343.00,1.00,4585.00,1.00,4188.00,1.00,0.00,0.00,nan,nan,nan,nan,nan,nan,nan,nan
5,3240447216.65,ag2004,20:59:00.500,4418.00,30.00,1988100.00,177686.00,4413.00,1.00,4418.00,11.00,4402.00,15.00,4422.00,5.00,4400.00,7.00,4424.00,3.00,4398.00,5.00,4429.00,15.00,4394.00,8.00,4434.00,8.00
6,3240447216.66,ag2005,20:59:00.500,4431.00,0.00,0.00,232.00,4382.00,1.00,4524.00,1.00,4188.00,1.00,4585.00,1.00,4173.00,1.00,4649.00,1.00,4169.00,1.00,0.00,0.00,nan,nan,nan,nan
7,3240447216.66,ag2006,20:59:00.500,4413.00,430.00,28463850.00,163504.00,4413.00,10.00,4440.00,1.00,4411.00,5.00,4443.00,3.00,4410.00,1.00,4444.00,2.00,4408.00,3.00,4445.00,20.00,4405.00,1.00,4446.00,24.00
8,3240447216.66,ag2007,20:59:00.500,4441.00,0.00,0.00,190.00,4408.00,1.00,4550.00,1.00,4377.00,1.00,4567.00,1.00,4228.00,1.00,4638.00,1.00,4197.00,1.00,0.00,0.00,nan,nan,nan,nan
9,3240447216.66,ag2008,20:59:00.500,4455.00,0.00,0.00,756.00,4440.00,1.00,4493.00,1.00,4420.00,1.00,4513.00,1.00,4400.00,1.00,4533.00,1.00,4380.00,1.00,4553.00,1.00,4360.00,1.00,4573.00,1.00


There are some data cleaning procedures before we continue our analysis. 
1. The time is not in datetime format so we need to convert it to that. 
2. Filtering out option contracts because we are not comparing them in this project
3. Obtain only for Active contracts for the highest volume
4. NaN values need to be removed

In [106]:
import datetime as dt
pd.set_option('display.float_format', '{:.2f}'.format) #Set display decimals


#Convert datetime object as Index
df = df.sort_values(['time'], ascending=True) #Sort time
df.time  = pd.to_datetime(df.time) #Convert time to datetime object
df.time  = pd.to_datetime(df.time).dt.strftime('%H:%M:%S.%f')##Remove Y/M/D , I only want H/M/S 

#obtain time from 09:00 - 15:00, Not including night session
df = df[(df.time >= '08:59:00.000') & (df.time <='15:00:00.500')]
df = df.set_index('time')#Set time as index

#Obtaining Active contract names 
#df = df[['Contract','volume','turnover', 'BP1','BV1','AP1','AV1']]
#filter out option contracts, more than 6 characters are options contracts
df = df[df['Contract'].str.len() <= 6]
#Filter in Active Contract, all contracts doesn't work...
df['Contract1'] = df.Contract.str.slice(start=0,stop =2)
df['Contract2'] = df.Contract.str.slice(start=2)
temp = df.groupby('Contract').max().reset_index()
temp= temp[['volume','Contract1','Contract']]
temp = temp[temp.volume == temp.groupby('Contract1')['volume'].transform('max')]
main = temp.Contract.to_numpy()
df = df[df['Contract'].isin(main)] 
df = df.sort_values(['time','Contract'], ascending=True)

#Remove NaNs 
df.dropna(inplace = True)

df.sort_values(['Contract1'], ascending=True)

df.head(25)

,Timestamp,Contract,last_price,volume,turnover,open_interest,BP1,BV1,AP1,AV1,BP2,BV2,AP2,AV2,BP3,BV3,AP3,AV3,BP4,BV4,AP4,AV4,BP5,BV5,AP5,AV5,Contract1,Contract2
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,
08:59:00.500000,3283647182.29,wr2001,3660.00,0.00,0.00,30.00,3636.00,1.00,3690.00,1.00,3620.00,1.00,3720.00,1.00,3606.00,1.00,3760.00,1.00,3566.00,1.00,3786.00,1.00,3506.00,1.00,3880.00,2.00,wr,2001
09:00:00.500000,3283707180.75,ag1912,4352.00,663578.00,43350185370.00,811918.00,4351.00,110.00,4352.00,24.00,4350.00,1032.00,4353.00,5.00,4349.00,43.00,4354.00,62.00,4348.00,77.00,4355.00,18.00,4347.00,32.00,4356.00,451.00,ag,1912
09:00:00.500000,3283707180.77,al1912,13815.00,22590.00,1561775000.00,230674.00,13810.00,52.00,13815.00,13.00,13805.00,338.00,13820.00,68.00,13800.00,81.00,13825.00,29.00,13795.00,108.00,13830.00,59.00,13790.00,67.00,13835.00,186.00,al,1912
09:00:00.500000,3283707180.78,au1912,341.20,181304.00,61837871700.00,347730.00,341.05,7.00,341.20,15.00,341.00,6.00,341.25,53.00,340.95,4.00,341.30,5.00,340.90,15.00,341.35,10.00,340.85,102.00,341.40,8.00,au,1912
09:00:00.500000,3283707180.80,bu1912,2948.00,335662.00,9884005640.00,430862.00,2944.00,51.00,2948.00,2.00,2942.00,201.00,2950.00,47.00,2940.00,237.00,2952.00,158.00,2938.00,214.00,2954.00,149.00,2936.00,251.00,2956.00,254.00,bu,1912
09:00:00.500000,3283707180.82,cu1912,47420.00,40374.00,9583929000.00,228344.00,47410.00,2.00,47420.00,9.00,47400.00,59.00,47430.00,1.00,47390.00,12.00,47440.00,66.00,47380.00,56.00,47460.00,1.00,47370.00,17.00,47470.00,3.00,cu,1912
09:00:00.500000,3283707180.85,fu2001,2093.00,422360.00,8881604260.00,760108.00,2093.00,19.00,2094.00,110.00,2092.00,7.00,2095.00,16.00,2091.00,164.00,2096.00,3.00,2090.00,363.00,2097.00,30.00,2089.00,492.00,2098.00,15.00,fu,2001
09:00:00.500000,3283707180.86,hc2001,3378.00,130152.00,4391675200.00,635200.00,3376.00,76.00,3378.00,10.00,3375.00,10.00,3379.00,503.00,3374.00,39.00,3380.00,176.00,3373.00,83.00,3381.00,19.00,3372.00,79.00,3382.00,7.00,hc,2001
09:00:00.500000,3283707180.87,ni1912,133900.00,271812.00,36422559200.00,238168.00,133880.00,6.00,133950.00,53.00,133840.00,17.00,133970.00,8.00,133830.00,61.00,133980.00,1.00,133820.00,9.00,134000.00,3.00,133810.00,13.00,134010.00,1.00,ni,1912


# Step 2 : Factor Analysis
The key signal in this project is 'Volume Order Imbalance'(VOI) and Midprice. Let's compute it.

In [55]:
#Compute VOI, the signal
def VOI(df):
    delta_VB = [0];delta_VA = [0]
    t=0;u=0
    for t in range(1,len(df.index)):
        if df.BP1[t] < df.BP1[t-1]:
            delta_VB.append(0)
        elif df.BP1[t] == df.BP1[t-1]:
            delta_VB.append(df.BV1[t] - df.BV1[t-1])
        elif df.BP1[t] > df.BP1[t-1]:
            delta_VB.append(df.BV1[t])
    for u in range(1,len(df.index)):
        if df.AP1[t] < df.AP1[t-1]:
            delta_VA.append(df.AV1[t])
        elif df.AP1[t] == df.AP1[t-1]:
            delta_VA.append(df.AV1[t] - df.AV1[t-1])
        elif df.AP1[t] > df.AP1[t-1]:
            delta_VA.append(0)
    VOI = np.subtract(delta_VB,delta_VA)
    return VOI

#Mid price , or we can fully utilitize the order book, VWAP(next step)
def midprice(df):
    midprice = (df.BP1 +df.AP1)/2
    return midprice

#Bid Ask Spread
def BAS(df):
    return (df.AP1 - df.BP1)

### Statistical Analysis 
For time series data analysis, a few things need to be checked before we run the linear model. 

1. Normalization of midprice change
2. Autocorrelation a.k.a. Serial Correlation : Check similarity within time series
3. Stationarity (ADF test), Series can steadily accumulate positive returns. Reject Null Hypothesis if the series is stationary. 

Before we check those, we need to normalize the midprice change by their tick size since we are comparing data within contracts.

In [180]:
rb = df[df.Contract1== 'rb']
#Obtaining ticksize
def ticksize(df):
    x=df['BP1'].diff()
    x = x.replace(0, np.nan)
    x.dropna(inplace=True)
    return abs(x.value_counts().idxmax())

temp = df.groupby('Contract').apply(midprice).diff()/df.groupby('Contract').apply(ticksize)
df.groupby('Contract').apply(ticksize)

Contract
ag1912    1.0000
al1912    5.0000
au1912    0.0500
bu1912    2.0000
cu1912   10.0000
fu2001    1.0000
hc2001    1.0000
ni1912   10.0000
pb1912    5.0000
rb2001    1.0000
ru2001    5.0000
sn2001   10.0000
sp2001    2.0000
ss2002    5.0000
wr2001    1.0000
zn1912    5.0000
dtype: float64

Now have our change in midprice normalized against ticksize. We can conduct our analysis. The first one is **Correlation**. 

In [144]:
pd.set_option('display.float_format', '{:.4f}'.format)
def cor(df):
    cor = np.corrcoef(np.diff(midprice(df))/ticksize(df),VOI(df)[1:])[1,0]
    return cor

correlation_table = pd.DataFrame(df.groupby('Contract').apply(cor))
print(correlation_table)

              0
Contract       
ag1912   0.1921
al1912   0.1787
au1912   0.1960
bu1912   0.1670
cu1912   0.1262
fu2001   0.3462
hc2001   0.1892
ni1912   0.2678
pb1912   0.2379
rb2001   0.1627
ru2001   0.2208
sn2001   0.2340
sp2001   0.2228
ss2002   0.1233
wr2001   0.1190
zn1912   0.1899


### Autocorrelation 


In [164]:
#Autocorrelation coefficient for dM with lag from 0 to 5
def dM_autocor(df):
    temp = midprice(df).diff()/ticksize(df)
    autocor = [round(temp.autocorr(lag = i),2) for i in range(0,6)]
    return autocor

autocorrr = df.groupby('Contract').apply(dM_autocor)
autocorrr

Contract
ag1912    [1.0, -0.11, -0.05, -0.03, -0.01, -0.02]
al1912     [1.0, -0.07, -0.1, -0.08, -0.01, -0.03]
au1912     [1.0, -0.19, -0.06, -0.04, -0.03, -0.0]
bu1912    [1.0, -0.08, -0.08, -0.02, -0.01, -0.02]
cu1912    [1.0, -0.18, -0.04, -0.01, -0.03, -0.03]
fu2001    [1.0, -0.11, -0.05, -0.02, -0.04, -0.01]
hc2001    [1.0, -0.23, -0.03, -0.01, -0.01, -0.01]
ni1912         [1.0, -0.2, 0.02, 0.03, 0.01, -0.0]
pb1912    [1.0, -0.16, -0.02, -0.05, -0.02, -0.02]
rb2001    [1.0, -0.14, -0.04, -0.01, -0.02, -0.02]
ru2001    [1.0, -0.05, -0.06, -0.04, -0.02, -0.01]
sn2001      [1.0, -0.18, 0.05, -0.01, 0.02, -0.01]
sp2001      [1.0, -0.16, -0.05, 0.02, -0.05, 0.02]
ss2002     [1.0, -0.26, -0.05, 0.06, -0.01, -0.03]
wr2001      [1.0, -0.01, -0.09, 0.01, -0.17, -0.0]
zn1912     [1.0, -0.18, -0.06, -0.0, -0.01, -0.02]
dtype: object

In [165]:
#Autocorrelation coefficient for VOI with lag from 0 to 5
def VOI_autocor(df):
    temp = VOI(df).diff()/ticksize(df)
    autocor = [round(temp.autocorr(lag = i),2) for i in range(0,6)]
    return autocor

autocorrr = df.groupby('Contract').apply(dM_autocor)
autocorrr

Contract
ag1912    [1.0, -0.11, -0.05, -0.03, -0.01, -0.02]
al1912     [1.0, -0.07, -0.1, -0.08, -0.01, -0.03]
au1912     [1.0, -0.19, -0.06, -0.04, -0.03, -0.0]
bu1912    [1.0, -0.08, -0.08, -0.02, -0.01, -0.02]
cu1912    [1.0, -0.18, -0.04, -0.01, -0.03, -0.03]
fu2001    [1.0, -0.11, -0.05, -0.02, -0.04, -0.01]
hc2001    [1.0, -0.23, -0.03, -0.01, -0.01, -0.01]
ni1912         [1.0, -0.2, 0.02, 0.03, 0.01, -0.0]
pb1912    [1.0, -0.16, -0.02, -0.05, -0.02, -0.02]
rb2001    [1.0, -0.14, -0.04, -0.01, -0.02, -0.02]
ru2001    [1.0, -0.05, -0.06, -0.04, -0.02, -0.01]
sn2001      [1.0, -0.18, 0.05, -0.01, 0.02, -0.01]
sp2001      [1.0, -0.16, -0.05, 0.02, -0.05, 0.02]
ss2002     [1.0, -0.26, -0.05, 0.06, -0.01, -0.03]
wr2001      [1.0, -0.01, -0.09, 0.01, -0.17, -0.0]
zn1912     [1.0, -0.18, -0.06, -0.0, -0.01, -0.02]
dtype: object

We can see autocorrelation for both exogenous and endogenous variable all vanish to zero as lag increases. Hence it shows that the time series is persistent.

### Stationarity
The first values gives the ADF test statistic, the second value returns the P-value. If P-value is 0.000 , that means reject Null hypothesis at 99%. Thus the time series is stationary.

In [176]:
from statsmodels.tsa.stattools import adfuller
#For dM
def dM_adf_test(df):
    x = midprice(df).diff()[1:]
    result = adfuller(x)
    l = [round(result[0],4), round(result[1],4)]
    return l

df.groupby('Contract').apply(dM_adf_test)

Contract
ag1912    [-44.1018, 0.0]
al1912    [-24.8968, 0.0]
au1912    [-26.8083, 0.0]
bu1912    [-36.6114, 0.0]
cu1912    [-37.5743, 0.0]
fu2001    [-34.5356, 0.0]
hc2001    [-43.0506, 0.0]
ni1912    [-51.5021, 0.0]
pb1912    [-39.2873, 0.0]
rb2001    [-48.0519, 0.0]
ru2001    [-45.0737, 0.0]
sn2001    [-64.3324, 0.0]
sp2001    [-21.3623, 0.0]
ss2002    [-48.2938, 0.0]
wr2001     [-9.5911, 0.0]
zn1912    [-32.0288, 0.0]
dtype: object

In [179]:
#For VOI
def VOI_adf_test(df):
    x = VOI(df)
    result = adfuller(x)
    l = [round(result[0],4), round(result[1],4)]
    return l
df.groupby('Contract').apply(VOI_adf_test)

Contract
ag1912     [-83.678, 0.0]
al1912    [-26.6563, 0.0]
au1912    [-65.1832, 0.0]
bu1912    [-20.2048, 0.0]
cu1912    [-48.9369, 0.0]
fu2001    [-98.1595, 0.0]
hc2001     [-24.282, 0.0]
ni1912    [-35.6773, 0.0]
pb1912    [-59.6626, 0.0]
rb2001    [-20.5119, 0.0]
ru2001    [-34.2519, 0.0]
sn2001    [-13.8988, 0.0]
sp2001    [-31.3668, 0.0]
ss2002    [-53.8318, 0.0]
wr2001     [-5.7293, 0.0]
zn1912    [-36.8564, 0.0]
dtype: object

# Step 4 : Linear Modelling

In the next step I ran the linear OLS model to check how well does the model fit the snapshot data. We are separating the approach to a few stages : 
1. Benchmark Model (Random Walk)
$$ \Delta M_t = \alpha + \epsilon _t $$
2. Fit VOI as exogenous variable 
$$ \Delta M_t = \alpha + \beta OI_t + \epsilon  _t $$
3. Multifactor Model
$$ \Delta M_t = \alpha + \beta_{OI} \frac{OI_t}{S_t} + \beta_{OIR} \frac{OIR_t}{S_t} + \beta_{MPR} \frac{MPR_t}{S_t} + \epsilon  _t $$


### Random Walk Model (Benchmark)
In the following results, the following sequence of number represents as:

$$[ \alpha, t(\alpha)  , R^2 , Adj  R^2 ]$$

The MSE (Mean Squared Error) can specified as:
$$ MSE = \frac{1}{n} \sum_{k=1}^{n} (y - y_{pred})^2 $$

In [230]:
def ols(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = np.array([1]*(len(df)-1))
    res = sm.OLS(y,x).fit()
    liss = [round(res.params[0],4),round(res.tvalues[0],2), round(res.rsquared,4), round(res.rsquared_adj,4)]
    return liss

model = df.groupby('Contract').apply(ols)
print(model)

Contract
ag1912      [0.0006, 0.43, -0.0, -0.0]
al1912        [0.0005, 0.32, 0.0, 0.0]
au1912           [0.0, 0.02, 0.0, 0.0]
bu1912        [0.0006, 0.65, 0.0, 0.0]
cu1912    [-0.0006, -0.36, -0.0, -0.0]
fu2001        [0.0006, 0.57, 0.0, 0.0]
hc2001      [-0.0012, -0.58, 0.0, 0.0]
ni1912      [-0.0009, -0.15, 0.0, 0.0]
pb1912    [-0.0017, -0.52, -0.0, -0.0]
rb2001      [-0.0005, -0.35, 0.0, 0.0]
ru2001      [-0.0001, -0.11, 0.0, 0.0]
sn2001      [0.0013, 0.11, -0.0, -0.0]
sp2001       [-0.0008, -0.7, 0.0, 0.0]
ss2002       [-0.001, -0.11, 0.0, 0.0]
wr2001         [0.007, 0.07, 0.0, 0.0]
zn1912    [-0.0003, -0.17, -0.0, -0.0]
dtype: object


In [231]:
#Compute MSE as Benchmark
def ols_mse(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = np.array([1]*(len(df)-1))
    res = sm.OLS(y,x).fit()
    pred = res.predict(x)
    return np.mean((y - pred)**2)

mse = df.groupby('Contract').apply(ols_mse)
print(mse)

Contract
ag1912   0.0495
al1912   0.0179
au1912   0.0594
bu1912   0.0170
cu1912   0.0395
fu2001   0.0247
hc2001   0.0860
ni1912   0.8353
pb1912   0.0859
rb2001   0.0516
ru2001   0.0361
sn2001   0.9560
sp2001   0.0174
ss2002   0.2550
wr2001   9.8046
zn1912   0.0498
dtype: float64


### VOI model
The following results are as follows:

$$[ \alpha, t(\alpha)  , \beta , t(\beta),  R^2 , Adj  R^2 ]$$


In [232]:
#Compute alpha, t-stats, beta, t-stats, R^2, adjusted R^2
def ols1(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = VOI(df)[1:]
    x = sm.add_constant(x)
    res = sm.OLS(y,x).fit()
    liss = [round(res.params[0],4),round(res.tvalues[0],2),round(res.params[1],4),
            round(res.tvalues[1],2), round(res.rsquared,2), round(res.rsquared_adj,2)]
    return liss

model1 = df.groupby('Contract').apply(ols1)
model1

Contract
ag1912     [-0.0099, -6.96, 0.0019, 30.9, 0.04, 0.04]
al1912      [-0.0125, -7.72, 0.002, 16.8, 0.03, 0.03]
au1912    [-0.0031, -1.72, 0.0048, 26.23, 0.04, 0.04]
bu1912     [-0.002, -2.26, 0.0005, 24.92, 0.03, 0.03]
cu1912    [-0.0019, -1.09, 0.0019, 14.16, 0.02, 0.02]
fu2001     [-0.0033, -3.5, 0.0018, 57.82, 0.12, 0.12]
hc2001    [-0.0081, -4.17, 0.0024, 28.65, 0.04, 0.04]
ni1912      [-0.23, -30.59, 0.0317, 44.6, 0.07, 0.07]
pb1912    [-0.0058, -1.87, 0.0159, 22.37, 0.06, 0.06]
rb2001    [-0.0099, -7.03, 0.0004, 27.04, 0.03, 0.03]
ru2001    [-0.0033, -2.56, 0.0036, 32.86, 0.05, 0.05]
sn2001      [0.1966, 13.53, 0.1275, 20.7, 0.05, 0.05]
sp2001    [-0.0007, -0.65, 0.0026, 25.93, 0.05, 0.05]
ss2002       [0.0056, 0.59, 0.0099, 6.66, 0.02, 0.01]
wr2001         [0.902, 3.26, 0.5177, 3.5, 0.01, 0.01]
zn1912    [-0.0099, -5.69, 0.0036, 25.04, 0.04, 0.04]
dtype: object

In [233]:
#Compute MSE 
def ols_mse1(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = VOI(df)[1:]
    x = sm.add_constant(x)
    res = sm.OLS(y,x).fit()
    pred = res.predict(x)
    return np.mean((y - pred)**2)

mse1 = df.groupby('Contract').apply(ols_mse1)
print(mse1)

Contract
ag1912   0.0477
al1912   0.0173
au1912   0.0571
bu1912   0.0165
cu1912   0.0389
fu2001   0.0217
hc2001   0.0829
ni1912   0.7754
pb1912   0.0811
rb2001   0.0503
ru2001   0.0343
sn2001   0.9036
sp2001   0.0166
ss2002   0.2511
wr2001   9.6657
zn1912   0.0480
dtype: float64


### Multifactor Model
We first compute factor Order Imbalance Ratio, Mean Reversion of Midprice and Bid Ask Spread for our Extended Model.

$$[ \alpha, t(\alpha)  , \beta_{OI} , t(\beta_{OI}),\beta_{OIR} , t(\beta_{OIR}),\beta_{MPR} , t(\beta_{MPR}),  R^2 , Adj  R^2 ]$$

In [263]:
#Order Imbalance Ratio
def OIR(df):
    return ((df.BV1 - df.AV1)/(df.BV1 + df.AV1))

#Mid price Reversion
def MPR(df):
    tp = [0]*len(df.index)
    mp = midprice(df)
    tp[0] = midprice(df)[0]
    for t in range(1, len(df.index)):
        if df.volume[t] != df.volume[t-1]:
            tp[t] = (df.turnover[t]-df.turnover[t-1])/((df.volume[t]-df.volume[t-1])*10)
        elif df.volume[t] == df.volume[t-1]:
            tp[t] = tp[t-1]
    return (tp - mp)


def ols2(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = pd.DataFrame(VOI(df)/BAS(df))
    x['OIR'] = OIR(df)/BAS(df)
    x['MPR'] = MPR(df)/BAS(df)
    x = sm.add_constant(x)
    res = sm.OLS(y,x[1:]).fit()
    liss = [round(res.params.const,3),round(res.tvalues.const,2),
            round(res.params[0],3),round(res.tvalues[0],2), 
            round(res.params.OIR,3),round(res.tvalues.OIR,2), 
            round(res.params.MPR,3),round(res.tvalues.MPR,2),
            round(res.rsquared,2), round(res.rsquared_adj,2)]
    return liss

model2 = df.groupby('Contract').apply(ols2)
model2


Contract
ag1912    [-0.041, -1.51, 0.002, 33.22, -0.15, -53.43, 0...
al1912    [-0.015, -0.43, 0.01, 17.16, -0.373, -22.87, -...
au1912    [-0.001, -0.04, 0.0, 27.89, -0.008, -45.2, -0....
bu1912    [0.001, 0.64, 0.001, 26.58, -0.147, -32.89, -0...
cu1912    [-0.009, -0.37, 0.02, 15.85, -1.19, -33.56, -0...
fu2001    [0.001, 1.13, 0.002, 58.42, -0.089, -34.82, -0...
hc2001    [-0.002, -1.07, 0.003, 32.56, -0.142, -42.41, ...
ni1912    [0.021, 1.36, 0.375, 46.93, -5.496, -48.05, 0....
pb1912    [-0.029, -1.53, 0.086, 25.02, -0.995, -35.31, ...
rb2001    [-0.011, -8.16, 0.0, 31.38, -0.118, -44.73, -0...
ru2001    [-0.003, -2.34, 0.019, 36.16, -0.923, -49.87, ...
sn2001    [0.079, 3.68, 3.236, 25.5, -5.835, -9.58, -0.0...
sp2001    [-0.006, -5.76, 0.005, 27.24, -0.108, -21.59, ...
ss2002    [-0.041, -1.72, 0.054, 7.31, -1.675, -14.54, -...
wr2001    [0.524, 2.78, 11.614, 4.81, -14.804, -1.37, -1...
zn1912    [-0.026, -0.97, 0.018, 26.31, -0.851, -49.56, ...
dtype: object

In [264]:
#Compute MSE 
def ols_mse2(df):
    y = midprice(df).diff()[1:]/ticksize(df)
    x = pd.DataFrame(VOI(df)/BAS(df))
    x['OIR'] = OIR(df)/BAS(df)
    x['MPR'] = MPR(df)/BAS(df)
    x = sm.add_constant(x)
    res = sm.OLS(y,x[1:]).fit()
    pred = res.predict(x)
    return np.mean((y - pred)**2)

mse2 = df.groupby('Contract').apply(ols_mse2)
print(mse2)

Contract
ag1912   0.0428
al1912   0.0163
au1912   0.0511
bu1912   0.0154
cu1912   0.0357
fu2001   0.0200
hc2001   0.0712
ni1912   0.7200
pb1912   0.0705
rb2001   0.0435
ru2001   0.0295
sn2001   0.8770
sp2001   0.0156
ss2002   0.2338
wr2001   9.4561
zn1912   0.0419
dtype: float64


In [261]:
print(res.params)
print(res.params.MPR)

const   -0.0107
0        0.0004
OIR     -0.1178
MPR     -0.0904
dtype: float64
-0.09038495927828431
